_Task 1: Inner Workings of ResNet-152_

_2. Residual Connections_

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet152

Using pre-trained ResNet-152.
Replacing the final classification layer to match CIFAR-10.
Freezing the rest of the backbone so that only the classification head is trained.
Disable skip connections in a few selected residual blocks and re-train the modified network head

In [ ]:
def remove_skip(block):
    orig_forward = block.forward

    def forward_no_skip(x):
        identity = torch.zeros_like(x)
        out = block.conv1(x)
        out = block.bn1(out)
        out = block.relu(out)

        out = block.conv2(out)
        out = block.bn2(out)
        out = block.relu(out)

        out = block.conv3(out)
        out = block.bn3(out)

        if block.downsample is not None:
            identity = block.downsample(x)

        out = out  # <-- skip removed
        out = block.relu(out)
        return out

    block.forward = forward_no_skip

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet152(weights='IMAGENET1K_V1')
remove_skip(model.layer3[0])
remove_skip(model.layer3[1])
remove_skip(model.layer4[0])
remove_skip(model.layer4[1])
model.fc = nn.Linear(model.fc.in_features, 10)
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
model = model.to(device)
print(device)

cuda


Preparing dataloaders

In [72]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform
)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=128, shuffle=False, num_workers=2
)

In [74]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.02, momentum=0.9)

In [75]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / total
    accuracy = 100. * correct / total
    return avg_loss, accuracy


def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / total
    accuracy = 100. * correct / total
    return avg_loss, accuracy

In [76]:
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, trainloader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, testloader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")

Epoch 1/5
  Train Loss: 0.6217 | Train Acc: 78.81%
  Val   Loss: 0.5318 | Val   Acc: 81.91%
Epoch 2/5
  Train Loss: 0.5143 | Train Acc: 82.58%
  Val   Loss: 0.4815 | Val   Acc: 83.66%
Epoch 3/5
  Train Loss: 0.4902 | Train Acc: 83.28%
  Val   Loss: 0.5057 | Val   Acc: 83.12%
Epoch 4/5
  Train Loss: 0.4656 | Train Acc: 83.84%
  Val   Loss: 0.4978 | Val   Acc: 82.97%
Epoch 5/5
  Train Loss: 0.4552 | Train Acc: 84.26%
  Val   Loss: 0.4601 | Val   Acc: 84.40%
